# 收获和价值的计算(马尔可夫奖励过程)
> 相当于MDP问题给定了策略$\pi$ 然后策略就相当于P 来求解
- 收获(return) 收获是对应于状态序列中的某一时刻的 状态的，计算从该状态开始直至结束还能获得的累积奖励 
$$G _ { t } = R _ { t + 1 } + \gamma R _ { t + 2 } + \ldots = \sum _ { k = 0 } ^ { \infty } \gamma ^ { k } R _ { t + k + 1 }$$
- 价值(value) 是马尔科夫奖励过程中状态收获的期望
$$ v ( s ) = E [ G _ { t } | S _ { t } = s ] $$
- 价值函数展开一窥奥秘
$$  \left. \begin{array}{l}{ v ( s ) } \\ { = E [ G _ { t } | S _ { t } = s ] }\\{ = E [ R _ { t + 1 } + \gamma v ( S _ { t + 1 } ) | S _ { t } = s ] } \\ { = R_s + \gamma v( s ^ { \prime } ) }\end{array} \right.$$
- 价值函数=>(贝尔曼方程) 等于此时刻的奖励值和下一个状态的价值。下一个状态的价值就是透过此时刻到下一时刻全部状态的平均价值
$$ v ( s ) = R _ { s } + \gamma \sum _ { s ^ { \prime } \in S } P _ { s s ^ { \prime } } v ( s ^ { \prime } ) $$
- 用矩阵形式计算价值。可以直接求解。复杂度O(n^3)
$$ \left. \begin{array}{l}{ v = R + \gamma P v }\\{ ( 1 - \gamma P ) v = R }\\{ v = ( 1 - \gamma P ) ^ { - 1 } R }\end{array} \right. $$
> 相较于值计算Return， Return只考虑了某些特殊的情况，并没有从必然性来考虑。所以求他的期望是比较准确的
> 这种马尔可夫奖励过程不涉及选择动作，预定了的概率存在，我们只是计算出了每种状态的价值->改进为马尔可夫决策过程(MDP)


In [2]:


import numpy as np
# 这些状态有:第 一节课(C1)、第二节课(C2)、第三节课(C3)、泡吧中(Pub)、通过考试(Pass)、浏览手机 (FB)、以及休息退出(Sleep)共 7 个状态
# 索引到状态名的对应

i_to_n = {}
i_to_n["0"] = "C1"
i_to_n["1"] = "C2"
i_to_n["2"] = "C3"
i_to_n["3"] = "Pass"
i_to_n["4"] = "Pub"
i_to_n["5"] = "FB"
i_to_n["6"] = "Sleep"

n_to_i={}# 状态名到索引的字典
for i, name in zip(i_to_n.keys(), i_to_n.values()):
    n_to_i[name] = int(i)

# 此时我们预定有概率转移矩阵的，因为我们的目标仅仅是计算收获Return和价值Value
Pss=[# 状态转移概率矩阵
    [ 0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0 ],
    [ 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.2 ],
    [ 0.0, 0.0, 0.0, 0.6, 0.4, 0.0, 0.0 ],
    [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0 ],
    [ 0.2, 0.4, 0.4, 0.0, 0.0, 0.0, 0.0 ],
    [ 0.1, 0.0, 0.0, 0.0, 0.0, 0.9, 0.0 ],
    [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0 ]
]

Pss = np.array(Pss)
# 奖励值、对应的是状态。也就是每个状态给他一个奖励值
rewards = [-2, -2, -2, 10, 1, -1 ,0]
gamma = 0.5

# 计算某一条链的累积奖励值Return
def compute_return(start_index = 0, chain = None, gamma = 0.5):
    retrn, power, gamma = 0.0, 0, gamma
    for i in range(start_index, len(chain)):
        retrn += np.power(gamma, power) * rewards[n_to_i[chain[i]]]
        power += 1
    return retrn

# 使用矩阵计算价值Value
# 相较于值计算Return， Return只考虑了某些特殊的情况，并没有从必然性来考虑。所以求他的期望是比较准确的
# 这种马尔可夫奖励过程不涉及选择动作，预定了的概率存在，我们只是计算出了每种状态的价值->改进为马尔可夫决策过程(MDP)
def compute_value(Pss, rewards, gamma = 0.999):
    # 将rewards转为numpy数组并修改为列向量的形式
    rewards = np.array(rewards).reshape(-1, 1)
    # np.eye(7,7)为 单 位 矩 阵，inv方 法 为 求 矩 阵 的 逆
    values = np.dot(np.linalg.inv(np.eye(7, 7) - gamma * Pss), rewards)
    return values


In [4]:
chains =[
["C1", "C2", "C3", "Pass", "Sleep"],
["C1", "FB", "FB", "C1", "C2", "Sleep"],
["C1", "C2", "C3", "Pub", "C2", "C3", "Pass", "Sleep"], ["C1", "FB", "FB", "C1", "C2", "C3", "Pub", "C1", "FB",\
"FB", "FB", "C1", "C2", "C3", "Pub", "C2", "Sleep"] ]

compute_return(0, chains[3], gamma=0.5)


-3.196044921875

In [6]:
compute_value(Pss, rewards, gamma=0.99999)

array([[-12.54073351],
       [  1.45690179],
       [  4.32117045],
       [ 10.        ],
       [  0.80308417],
       [-22.53857963],
       [  0.        ]])

# 马尔可夫决策过程

> 不同的$\pi$有不同的价值函数

- 采用行为才会获得奖励

- 策略(Policy) 个体有根据自身对当前状态的认识从行为集中选择 一个行为的权利，而个体在选择某一个行为后其后续状态则由环境的动力学决定。从行为集合种选择一个行为。$\pi$是一个概率分布
$$ \pi ( a | s ) = P [ A _ { t } = a | S _ { t } = s ] $$

- 价值函数$v_{\pi}(s)$ 也就是收获的期望(是基于策略，因为概率不同我们最终的期望是不同的)
$$ v _ { \pi } ( s ) = E [ G _ { t } | S _ { t } = s ] $$

- 引入行为价值函数$q_{\pi}(s,a)$(知道我们采取行为的函数)表示在遵循策略$\pi$时，对当前状态 s 执行某一具体行为 a 所能的到 的收获的期望:
$$ q _ { \pi } ( s , a ) = E [ G _ { t } | S _ { t } = s , A _ { t } = a ] $$
$$ q _ { \pi } ( s , a ) = E [ R _ { t + 1 } + \gamma q _ { \pi } ( S _ { t + 1 } , A _ { t + 1 } ) | S _ { t } = s , A _ { t } = a ] $$

- 转换关系
$$ v _ { \pi } ( s ) = \sum _ { a \in A } \pi ( a | s ) q _ { \pi } ( s , a ) $$
$$q _ { \pi } ( s , a ) = R _ { s } ^ { a } + \gamma \sum _ { s ^ { \prime } \in S } P _ { s s ^ { \prime } } ^ { a } v _ { \pi } ( s ^ { \prime } )$$

> $\sum _ { s ^ { \prime } \in S } P _ { s s ^ { \prime } } ^ { a } v _ { \pi } ( s ^ { \prime } )$ 表示下一个状态的期望价值

- 转换关系组合起来 得到以下两种形式
$$v _ { \pi } ( s ) = \sum _ { a \in A } \pi ( a | s ) ( R _ { s } ^ { a } + \gamma \sum _ { s ^ { \prime } \in S } P _ { s s ^ { \prime } } ^ { a } v _ { \pi } ( s ^ { \prime } ) ) $$
$$ q _ { \pi } ( s , a ) = R _ { s } ^ { a } + \gamma \sum _ { s ^ { \prime } \in S } P _ { s s ^ { \prime } } ^ { a } \sum _ { a ^ { \prime } \in A } \pi ( a ^ { \prime } | s ^ { \prime } ) q _ { \pi } ( s ^ { \prime } , a ^ { \prime } )$$

## 最优行为价值函数、最优状态价值函数
- 最优状态价值函数$v _ { * } = \operatorname { max } _ { \pi } v _ { \pi } ( s )$
- 最优行为价值函数$q _ { * } ( s , a ) = \operatorname { max } _ { \pi } q _ { \pi } ( s , a )$

很明显最优策略最终会使行为价值某个行为概率趋于1 因此求解强化学习问题就转变为了求解最优行为价值函数问题。
$$ v _ { * } ( s ) = \operatorname { max } _ { a } q _ { * } ( s , a ) $$

- 贝尔曼最优方程 (也就是我们需要求解的内容)
$$ q _ { * } ( s , a ) = R _ { s } ^ { a } + \gamma \sum _ { s ^ { \prime } \in S } P _ { s s ^ { \prime } } ^ { a } v _ { * } ( s ^ { \prime } )$$
    1.执行该行为后环境给予的 确定的即时奖励
    2.所有后续可能状态的最优状态价值按发生概率求和乘以衰减系 数得到。也就是下一所有可能状态的期望价值

- 最终的求法(利用贝尔曼、以及整理成迭代形式)
$$ q _ { * } ( s , a ) = R _ { s } ^ { a } + \gamma \sum _ { s ^ { \prime } \in S } P _ { s s ^ { \prime } } ^ { a } \operatorname { max } _ { a ^ { \prime } } q _ { * } ( s ^ { \prime } , a ^ { \prime } ) $$
> 理解1:最优q值的计算。(s,a)的值由(s,a)->s' 进入下一时刻所有可能的s'。以及s'他们那个时刻能采取的最大行为价值

> 理解2:贝尔曼最优方程不是线性方程，无法直接求解，通常采用迭代法来求解，具体有价值迭代、 策略迭代、Q 学习、Sarsa 学习等多种迭代方法，

In [5]:
# 状态数变 成了 5 个，为了方便理解，我们把这五个状态分别命名为:‘浏览手机中’,‘第一节课’,‘第二 节课’,‘第三节课’,‘休息中’;
# 行为总数也是 5 个，但具体到某一状态则只有 2 个可能的行 为，这 5 个行为分别命名为:‘浏览手机’,‘学习’,‘离开浏览’,‘泡吧’,‘退出学习’

from utils import str_key, display_dict
# 设置转移概率、奖励值以及读取他们的方法
from utils import set_prob, set_reward, get_prob, get_reward
# 设置状态价值、策略概率以及读取他们
from utils import set_value, set_pi, get_value, get_pi

# 所有的State以及Action
S = ['浏览手机中','第一节课','第二节课','第三节课','休息中'] 
A = ['浏览手机','学习','离开浏览','泡吧','退出学习']

R={}# 奖励Rsa字典
# 这个P不是我们需要求的策略，而是确实存在的。有些行为进入下一个状态并不是固定的
P={}# 状态转移概率Pss'a字典 
gamma=1.0# 衰减因子

# 根据学生马尔科夫决策过程示例的数据设置状态转移概率和奖励，默认概率为1 
set_prob(P,S[0],A[0],S[0])# 浏览手机中 - 浏览手机 -> 浏览手机中 
set_prob(P,S[0],A[2],S[1])# 浏览手机中 - 离开浏览 -> 第一节课 
set_prob(P,S[1],A[0],S[0])# 第一节课 - 浏览手机 -> 浏览手机中 
set_prob(P,S[1],A[1],S[2])# 第一节课 - 学习 -> 第二节课
set_prob(P,S[2],A[1],S[3])# 第二节课 - 学习 -> 第三节课 
set_prob(P,S[2],A[4],S[4])# 第二节课 - 退出学习 -> 退出休息 
set_prob(P,S[3],A[1],S[4])# 第三节课 - 学习 -> 退出休息 
set_prob(P,S[3],A[3],S[1],p=0.2)# 第三节课 - 泡吧 -> 第一节课 
set_prob(P,S[3],A[3],S[2],p=0.4)# 第三节课 - 泡吧 -> 第一节课 
set_prob(P,S[3],A[3],S[3],p=0.4)# 第三节课 - 泡吧 -> 第一节课

set_reward(R,S[0],A[0],-1)# 浏览手机中 - 浏览手机 ->-1 
set_reward(R,S[0],A[2], 0)# 浏览手机中 - 离开浏览 ->0 
set_reward(R,S[1],A[0],-1)# 第一节课 - 浏览手机 ->-1 
set_reward(R,S[1],A[1],-2)# 第一节课 - 学习 ->-2 
set_reward(R,S[2],A[1],-2)# 第二节课 - 学习 ->-2
set_reward(R,S[2],A[4], 0)# 第二节课 - 退出学习 ->0 
set_reward(R,S[3],A[1],10)# 第三节课 - 学习 ->10 
set_reward(R,S[3],A[3],+1)# 第三节课 - 泡吧 ->-1

MDP=(S,A,R,P,gamma)

# 求策略

In [15]:
# 使用随机策略 如果下一个行为有两种我们就使用0.5概率
Pi = {}
set_pi(Pi,S[0],A[0],0.5)# 浏览手机中 - 浏览手机 
set_pi(Pi,S[0],A[2],0.5)# 浏览手机中 - 离开浏览
set_pi(Pi,S[1],A[0],0.5)# 第一节课 - 浏览手机 
set_pi(Pi,S[1],A[1],0.5)# 第一节课 - 学习 
set_pi(Pi,S[2],A[1],0.5)# 第二节课 - 学习 
set_pi(Pi,S[2],A[4],0.5)# 第二节课 - 退出学习 
set_pi(Pi,S[3],A[1],0.5)# 第三节课 - 学习 
set_pi(Pi,S[3],A[3],0.5)# 第三节课 - 泡吧

# 初始价值为空
V = {}

In [19]:
# 在给定MDP和V的条件下计算 某一状态s时才去行为a的价值q(s,a)

def compute_q(MDP, V, s, a):
    S, A, R, P, gamma = MDP
    q_sa = 0
    for s_prime in S:
        q_sa += get_prob(P, s,a,s_prime) * get_value(V, s_prime)
    q_sa = get_reward(R, s,a) + gamma * q_sa
    return q_sa

# 给定策略Pi下如何计算某一状态的价值
def compute_v(MDP, V, Pi, s):
    S, A, R, P, gamma = MDP
    v_s = 0
    for a in A:
        v_s += get_pi(Pi, s, a) * compute_q(MDP, V, s, a)
    return v_s

# 根据当前策略使用回溯法来更新状态价值 更新了所有的Vs
def update_V(MDP, V, Pi):
    S, _, _, _, _ = MDP
    V_prime = V.copy()

    for s in S:
        V_prime[str_key(s)] = compute_v(MDP, V_prime, Pi, s)
    
    return V_prime

# 策略评估，得到该策略下最终的状态价值。
def policy_evaluate(MDP, V, Pi, n):
    # 使 用n次 迭 代 计 算 来 评 估 一 个MDP在 给 定 策 略Pi下 的 状 态 价 值 ， 初 始 时 价 值 为V

    for i in range(n):
        V = update_V(MDP, V, Pi)
    return V

# 计算最优策略下最优状态价值
def compute_v_from_max_q(MDP, V, s):
    S, A, R, P, gamma = MDP
    v_s = -float('inf')
    for a in A:
        qsa = compute_q(MDP, V, s, a)
        if qsa >= v_s:
            v_s = qsa
    return v_s

# 在不依赖策略的情况下直接通过后续状态的价值来更新状态价值
def update_V_without_pi(MDP, V):
    S, _, _, _, _ = MDP
    V_prime = V.copy()
    for s in S:
        #set_value(V_prime, s, compute_v_from_max_q(MDP, V_prime, s))
        V_prime[str_key(s)] = compute_v_from_max_q(MDP, V_prime, s) 
    return V_prime

# 价值迭代
def value_iterate(MDP, V, n): 
    for i in range(n):
        V = update_V_without_pi(MDP, V)
    return V

In [17]:
V = policy_evaluate(MDP, V, Pi, 100)

In [18]:
V

{'浏览手机中': -2.307692307700891,
 '第一节课': -1.3076923076969194,
 '第二节课': 2.6923076923072005,
 '第三节课': 7.384615384614628,
 '休息中': 0.0}